In [38]:
import pandas as pd
import numpy as np
from utils import load_model, get_prediction_for_session
import gensim
from gensim.models import Word2Vec
import gc
from metrics import map_at_k, mean_ndcg

In [2]:
cart_model = load_model("pretrained_models/cart.model")
view_model = load_model("pretrained_models/view.model")

In [3]:
df = pd.read_json('../non_train_data/processed.json')

In [ ]:
from tqdm import tqdm_notebook
predictions = []
actual = []
for i, (view, cart, order) in tqdm_notebook(df[:1000].iterrows()):
    session = {'views': view, 'to_cart': cart}
    prediction = get_prediction_for_session(view_model, cart_model, session, n=20)
    predictions.append(prediction)
    actual.append(order)

In [42]:
def avg_precision(actual, predicted, k = 10):
    k = min(k, len(actual))
    score = 0.0
    true_predictions = 0.0
    for i,predict in enumerate(predicted[:k]):
        if predict in actual:
            true_predictions += 1.0
            score += true_predictions / (i+1.0)
    return score / k


def map_at_k(df: pd.DataFrame, k = 10):
    true, predictions = df.values.T
    ap_at_k = [avg_precision(act, pred, k) for act, pred in zip(true, predictions)]
    return np.mean(ap_at_k)

In [34]:
data = pd.read_csv('../non_train_data/sessions.csv', index_col=0)
data.head()

cart_data = data[(data.action_type == 'to_cart')]
from collections import Counter
cntr = Counter(cart_data.item_id.values)

tops = [str(i[0]) for i in cntr.most_common(40)]
naive_prediction = [tops] * 1000

/home/huvi/anaconda3/envs/howto/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [43]:
to_evaluate = pd.DataFrame({'actual':actual, 'preds':naive_prediction})
print(mean_ndcg(to_evaluate, k=40))
print(map_at_k(to_evaluate, k=40))

0.00017291568352954326
0.0005676609598604483


In [44]:
to_evaluate = pd.DataFrame({'actual':actual, 'preds':predictions})
print(mean_ndcg(to_evaluate, k=40))
print(map_at_k(to_evaluate, k=40))

0.0018028898698115544
0.015233346949891067
